In [7]:
from jax_circuits import*
from topology import *
from penalty import *
from cp_utils import *
import matplotlib.pyplot as plt
from scipy.stats import unitary_group
from qiskit import transpile
from qiskit.quantum_info import Operator
import pickle
import time

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [8]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

def objective(args):
    r, num_gates = args
    num_gates = int(num_gates)
    anz = Ansatz(3, 'cp', placements=fill_layers(sequ_layer(3), num_gates))
    reg_options = {'r': r,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'accepted_num_gates': 10,
               'target_num_gates': 6,
               'cp_mask': anz.cp_mask}
    
    time_start = time.time()
    score, cz_counts = cp_ansatz_score(u_toff3, 
                                       anz, 
                                       reg_options, 
                                       num_samples=20,
                                       save_prospective_results=False,
                                       save_raw_results=False)
    eval_time = time.time()-time_start
    
    return {
        'loss': -score,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': eval_time,
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }    
    

In [9]:
space = [hp.uniform('r', 0.0001, 0.01),
        hp.quniform('num_gates', 6, 30, 1)]

trials = Trials()
best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials)

print(best)

100%|██████████| 10/10 [05:11<00:00, 31.19s/trial, best loss: -7.1875]
{'num_gates': 11.0, 'r': 0.007872887239086376}
